# 강남구 건축물현황 최종본

In [45]:
import os
import numpy as np
import pandas as pd
import requests
import json
import pprint
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import precision_recall_curve, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from geopy.geocoders import Nominatim

from  sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from scipy.stats import boxcox
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [46]:
dfgn = pd.read_csv("../../강남구_최종.csv")
dfgn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24025 entries, 0 to 24024
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   대지위치        24025 non-null  object 
 1   대지면적(㎡)     24025 non-null  float64
 2   건축면적(㎡)     24025 non-null  float64
 3   건폐율(%)      24025 non-null  float64
 4   연면적(㎡)      24025 non-null  float64
 5   용적률(%)      24025 non-null  float64
 6   구조          24025 non-null  int64  
 7   세대수(세대)     24025 non-null  int64  
 8   가구수(가구)     24025 non-null  int64  
 9   높이(m)       24025 non-null  float64
 10  지상층수        24025 non-null  int64  
 11  지하층수        24025 non-null  int64  
 12  승용승강기수      24025 non-null  int64  
 13  비상용승강기수     24025 non-null  int64  
 14  부속건축물수      24025 non-null  int64  
 15  부속건축물면적(㎡)  24025 non-null  float64
 16  총동연면적(㎡)    24025 non-null  float64
 17  노후화         24025 non-null  int64  
 18  위도          24025 non-null  float64
 19  경도          24025 non-nul

In [47]:
dfgn.drop(columns=['총동연면적(㎡)','높이(m)','승용승강기수','비상용승강기수',
                   '부속건축물수','부속건축물면적(㎡)','용적률(%)','건폐율(%)'
                  ,'대지면적(㎡)','세대수(세대)','가구수(가구)'], inplace=True)
dfgn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24025 entries, 0 to 24024
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   대지위치     24025 non-null  object 
 1   건축면적(㎡)  24025 non-null  float64
 2   연면적(㎡)   24025 non-null  float64
 3   구조       24025 non-null  int64  
 4   지상층수     24025 non-null  int64  
 5   지하층수     24025 non-null  int64  
 6   노후화      24025 non-null  int64  
 7   위도       24025 non-null  float64
 8   경도       24025 non-null  float64
dtypes: float64(4), int64(4), object(1)
memory usage: 1.6+ MB


In [50]:
dfgn['time_unit_tmprt'] = np.nan
dfgn['time_unit_ws'] = np.nan
dfgn['time_unit_wd'] = np.nan
dfgn['time_unit_humidity'] = np.nan
# 실시간 기상청 API가 nan 값을 채워주면서 동시에 예측이 가능

In [51]:
dfgn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24025 entries, 0 to 24024
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   대지위치                24025 non-null  object 
 1   건축면적(㎡)             24025 non-null  float64
 2   연면적(㎡)              24025 non-null  float64
 3   구조                  24025 non-null  int64  
 4   지상층수                24025 non-null  int64  
 5   지하층수                24025 non-null  int64  
 6   노후화                 24025 non-null  int64  
 7   위도                  24025 non-null  float64
 8   경도                  24025 non-null  float64
 9   time_unit_tmprt     0 non-null      float64
 10  time_unit_ws        0 non-null      float64
 11  time_unit_wd        0 non-null      float64
 12  time_unit_humidity  0 non-null      float64
dtypes: float64(8), int64(4), object(1)
memory usage: 2.4+ MB


In [52]:
dfgn.rename(columns={
    '건축면적(㎡)': 'bottom_area',
    '연면적(㎡)': 'totar',
    '지상층수': 'ground_nof',
    '지하층수': 'bstory_cnt',
    '위도': 'lat',
    '경도': 'lon'
}, inplace=True)

In [53]:
dfgn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24025 entries, 0 to 24024
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   대지위치                24025 non-null  object 
 1   bottom_area         24025 non-null  float64
 2   totar               24025 non-null  float64
 3   구조                  24025 non-null  int64  
 4   ground_nof          24025 non-null  int64  
 5   bstory_cnt          24025 non-null  int64  
 6   노후화                 24025 non-null  int64  
 7   lat                 24025 non-null  float64
 8   lon                 24025 non-null  float64
 9   time_unit_tmprt     0 non-null      float64
 10  time_unit_ws        0 non-null      float64
 11  time_unit_wd        0 non-null      float64
 12  time_unit_humidity  0 non-null      float64
dtypes: float64(8), int64(4), object(1)
memory usage: 2.4+ MB


In [54]:
# 소방서와의 거리
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # 지구 반지름 (m)
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)

    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c  # meters

In [55]:
# 강남소방서 좌표
fire_lat = 37.51036
fire_lon = 127.0668895

# 건축물 데이터: 위도, 경도가 있는 df
# 컬럼명이 'latitude', 'longitude' 또는 'la', 'longitude' 라고 가정

dfgn['spt_frstt_dist'] = dfgn.apply(
    lambda row: haversine(row['lat'], row['lon'], fire_lat, fire_lon),
    axis=1
)
dfgn['spt_frstt_dist'] = (dfgn['spt_frstt_dist'] / 1000).round(1)

print(dfgn[['lat', 'lon', 'spt_frstt_dist']].tail())

             lat         lon  spt_frstt_dist
24020  37.512281  127.034913             2.8
24021  37.512244  127.034764             2.8
24022  37.512113  127.034817             2.8
24023  37.512150  127.034964             2.8
24024  37.509086  127.035930             2.7


In [56]:
dfgn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24025 entries, 0 to 24024
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   대지위치                24025 non-null  object 
 1   bottom_area         24025 non-null  float64
 2   totar               24025 non-null  float64
 3   구조                  24025 non-null  int64  
 4   ground_nof          24025 non-null  int64  
 5   bstory_cnt          24025 non-null  int64  
 6   노후화                 24025 non-null  int64  
 7   lat                 24025 non-null  float64
 8   lon                 24025 non-null  float64
 9   time_unit_tmprt     0 non-null      float64
 10  time_unit_ws        0 non-null      float64
 11  time_unit_wd        0 non-null      float64
 12  time_unit_humidity  0 non-null      float64
 13  spt_frstt_dist      24025 non-null  float64
dtypes: float64(9), int64(4), object(1)
memory usage: 2.6+ MB


# 아웃라이어 ( 존재하지 않는 주소 )
- 서울특별시 강남구 삼성동 627-13번지
- 서울특별시 강남구 압구정동 26-3번지
- 서울특별시 강남구 압구정동 189-3번지
- 서울특별시 강남구 압구정동 26-3번지
- 서울특별시 강남구 압구정동 26-3번지

In [57]:
삭제할주소 = [
    '서울특별시 강남구 삼성동 627-13번지',
    '서울특별시 강남구 압구정동 26-3번지',
    '서울특별시 강남구 압구정동 189-3번지',
    '서울특별시 강남구 압구정동 26-3번지',
    '서울특별시 강남구 압구정동 26-3번지'
]

dfgn = dfgn[~dfgn['대지위치'].isin(삭제할주소)]
dfgn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24020 entries, 0 to 24024
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   대지위치                24020 non-null  object 
 1   bottom_area         24020 non-null  float64
 2   totar               24020 non-null  float64
 3   구조                  24020 non-null  int64  
 4   ground_nof          24020 non-null  int64  
 5   bstory_cnt          24020 non-null  int64  
 6   노후화                 24020 non-null  int64  
 7   lat                 24020 non-null  float64
 8   lon                 24020 non-null  float64
 9   time_unit_tmprt     0 non-null      float64
 10  time_unit_ws        0 non-null      float64
 11  time_unit_wd        0 non-null      float64
 12  time_unit_humidity  0 non-null      float64
 13  spt_frstt_dist      24020 non-null  float64
dtypes: float64(9), int64(4), object(1)
memory usage: 2.7+ MB


In [58]:
# fire_data랑 똑같이 One-Hot Encoding

In [59]:
dfgn['구조'].unique()

array([7, 2, 6, 5, 0, 1, 4, 3])

In [60]:
dfgn['구조'] = dfgn['구조'].replace(0, 1)
dfgn['구조'].unique()

array([7, 2, 6, 5, 1, 4, 3])

In [61]:
mapping_dict = {
    '철근콘크리트구조': 7, '벽돌구조': 2, '일반목구조': 4, 
    '일반철골구조': 5, '기타': 1, '블록구조': 3, 
    '철골철근콘크리트구조': 6
}
reverse_mapping_dict = {v: k for k, v in mapping_dict.items()}
dfgn['구조명'] = dfgn['구조'].map(reverse_mapping_dict)

print("\n===== 1단계: 숫자 코드를 텍스트 라벨로 변환 후 =====")
print(dfgn)


# --- 2단계: 텍스트 라벨 컬럼을 원-핫 인코딩 ---

# pd.get_dummies()를 사용하여 '구조명' 컬럼을 원-핫 인코딩합니다.
# 이 과정에서 기존의 '구조명' 컬럼은 자동으로 삭제됩니다.
dfgn = pd.get_dummies(dfgn, columns=['구조명'], prefix='구조')


===== 1단계: 숫자 코드를 텍스트 라벨로 변환 후 =====
                         대지위치  bottom_area    totar  구조  ground_nof  \
0      서울특별시 강남구 역삼동 628-12번지       269.59  1956.35   7           8   
1      서울특별시 강남구 역삼동 628-13번지       562.99  4604.50   7           6   
2      서울특별시 강남구 역삼동 628-14번지       558.43  5077.38   7           8   
3      서울특별시 강남구 역삼동 628-15번지       152.18   830.32   7           7   
4         서울특별시 강남구 역삼동 629번지       135.53   657.35   7           4   
...                       ...          ...      ...  ..         ...   
24020  서울특별시 강남구 논현동 218-12번지       120.40   476.30   7           4   
24021  서울특별시 강남구 논현동 218-13번지       118.92   502.44   7           4   
24022  서울특별시 강남구 논현동 218-14번지       125.75   526.08   7           4   
24023  서울특별시 강남구 논현동 218-15번지       117.20   393.23   7           4   
24024   서울특별시 강남구 논현동 232-2번지       114.78   553.59   7           4   

       bstory_cnt  노후화        lat         lon  time_unit_tmprt  time_unit_ws  \
0               2    8  37.50

In [62]:
dfgn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24020 entries, 0 to 24024
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   대지위치                24020 non-null  object 
 1   bottom_area         24020 non-null  float64
 2   totar               24020 non-null  float64
 3   구조                  24020 non-null  int64  
 4   ground_nof          24020 non-null  int64  
 5   bstory_cnt          24020 non-null  int64  
 6   노후화                 24020 non-null  int64  
 7   lat                 24020 non-null  float64
 8   lon                 24020 non-null  float64
 9   time_unit_tmprt     0 non-null      float64
 10  time_unit_ws        0 non-null      float64
 11  time_unit_wd        0 non-null      float64
 12  time_unit_humidity  0 non-null      float64
 13  spt_frstt_dist      24020 non-null  float64
 14  구조_기타               24020 non-null  bool   
 15  구조_벽돌구조             24020 non-null  bool   
 16  구조_블록구조  

In [63]:
dfgn.drop(columns=['대지위치','구조','노후화'], inplace=True)
dfgn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24020 entries, 0 to 24024
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bottom_area         24020 non-null  float64
 1   totar               24020 non-null  float64
 2   ground_nof          24020 non-null  int64  
 3   bstory_cnt          24020 non-null  int64  
 4   lat                 24020 non-null  float64
 5   lon                 24020 non-null  float64
 6   time_unit_tmprt     0 non-null      float64
 7   time_unit_ws        0 non-null      float64
 8   time_unit_wd        0 non-null      float64
 9   time_unit_humidity  0 non-null      float64
 10  spt_frstt_dist      24020 non-null  float64
 11  구조_기타               24020 non-null  bool   
 12  구조_벽돌구조             24020 non-null  bool   
 13  구조_블록구조             24020 non-null  bool   
 14  구조_일반목구조            24020 non-null  bool   
 15  구조_일반철골구조           24020 non-null  bool   
 16  구조_철골철근콘크

In [64]:
# dfgn.drop(columns=['buld_strctr'], inplace=True)
dfgn.rename(columns={'구조_기타': 'strc_etc',
                     '구조_벽돌구조': 'strc_brick',
                     '구조_블록구조': 'strc_block',
                     '구조_일반목구조': 'strc_wood',
                     '구조_일반철골구조': 'strc_steel',
                     '구조_철골철근콘크리트구조': 'strc_src',
                     '구조_철근콘크리트구조': 'strc_rc'}, inplace=True)

dfgn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24020 entries, 0 to 24024
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bottom_area         24020 non-null  float64
 1   totar               24020 non-null  float64
 2   ground_nof          24020 non-null  int64  
 3   bstory_cnt          24020 non-null  int64  
 4   lat                 24020 non-null  float64
 5   lon                 24020 non-null  float64
 6   time_unit_tmprt     0 non-null      float64
 7   time_unit_ws        0 non-null      float64
 8   time_unit_wd        0 non-null      float64
 9   time_unit_humidity  0 non-null      float64
 10  spt_frstt_dist      24020 non-null  float64
 11  strc_etc            24020 non-null  bool   
 12  strc_brick          24020 non-null  bool   
 13  strc_block          24020 non-null  bool   
 14  strc_wood           24020 non-null  bool   
 15  strc_steel          24020 non-null  bool   
 16  strc_src 

## 특정 컬럼 log1p 변환

In [65]:
log_cols = [
    'ground_nof', 'bstory_cnt', 'totar', 'bottom_area',
    'spt_frstt_dist'
]

# log1p 적용 (음수 방지 위해 clip)
for col in log_cols:
    dfgn[col] = np.log1p(dfgn[col].clip(lower=0))

In [66]:
dfgn.head()

,bottom_area,totar,ground_nof,bstory_cnt,lat,lon,time_unit_tmprt,time_unit_ws,time_unit_wd,time_unit_humidity,spt_frstt_dist,strc_etc,strc_brick,strc_block,strc_wood,strc_steel,strc_src,strc_rc
0,5.600605,7.579347,2.197225,1.098612,37.503944,127.035095,NaN,NaN,NaN,NaN,1.360977,False,False,False,False,False,False,True
1,6.335037,8.435007,1.945910,1.098612,37.503733,127.035120,NaN,NaN,NaN,NaN,1.360977,False,False,False,False,False,False,True
2,6.326918,8.532748,2.197225,1.098612,37.503499,127.035229,NaN,NaN,NaN,NaN,1.360977,False,False,False,False,False,False,True
3,5.031614,6.723015,2.079442,0.693147,37.504154,127.034695,NaN,NaN,NaN,NaN,1.360977,False,False,False,False,False,False,True
4,4.916544,6.489737,1.609438,0.693147,37.503112,127.033016,NaN,NaN,NaN,NaN,1.410987,False,False,False,False,False,False,True


In [67]:
dfgn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24020 entries, 0 to 24024
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bottom_area         24020 non-null  float64
 1   totar               24020 non-null  float64
 2   ground_nof          24020 non-null  float64
 3   bstory_cnt          24020 non-null  float64
 4   lat                 24020 non-null  float64
 5   lon                 24020 non-null  float64
 6   time_unit_tmprt     0 non-null      float64
 7   time_unit_ws        0 non-null      float64
 8   time_unit_wd        0 non-null      float64
 9   time_unit_humidity  0 non-null      float64
 10  spt_frstt_dist      24020 non-null  float64
 11  strc_etc            24020 non-null  bool   
 12  strc_brick          24020 non-null  bool   
 13  strc_block          24020 non-null  bool   
 14  strc_wood           24020 non-null  bool   
 15  strc_steel          24020 non-null  bool   
 16  strc_src 

In [68]:
dfgn.to_csv('강남구0622.csv', index=False, encoding='utf-8-sig')